In [2]:
import os
import re
import pandas as pd
import numpy as np
from scipy.stats import spearmanr

In [3]:
GPT4_path = '../calculated files/gpt_pairs_no_relation_'
ANS_PATH = '../data/raw/Testing/Phase2Answers'
REV_PATH = '../data/reversals.txt'
CAT_PATH = '../data/subcategories-list.txt'
REP_PATH = '../data/external/tabular_file.xlsx'

In [4]:
lst_gpt_dfs = []
lst_data_dfs = []
rel_mapping = {}
lst_files = os.listdir(GPT4_path)

for file_name in lst_files:
    idx = re.findall('[0-9]+[a-z]', file_name)[0]

    data = pd.read_csv(os.path.join(ANS_PATH, 'Phase2Answers-{}.txt'.format(idx)), sep='\t')
    data = data.rename({'# pair1': 'pair1'}, axis='columns')
    rel_name = data['user_selected_relation'].unique()[0]

    answers = pd.read_csv(os.path.join(GPT4_path, 'Phase2Answers-{}_no_least_most_gpt.txt'.format(idx)), sep=', ', names=['pair1', 'pair2', 'pair3', 'pair4', 'least_illustrative', 'most_illustrative'])
    answers['user_selected_relation'] = rel_name
    answers['idx'] = idx
    lst_gpt_dfs.append(answers)
    lst_data_dfs.append(data)

    rel_mapping[idx] = rel_name

answers = pd.concat(lst_gpt_dfs).reset_index(drop=True)
data = pd.concat(lst_data_dfs).reset_index(drop=True)

/tmp/ipykernel_4556/881429521.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  answers = pd.read_csv(os.path.join(GPT4_path, 'Phase2Answers-{}_no_least_most_gpt.txt'.format(idx)), sep=', ', names=['pair1', 'pair2', 'pair3', 'pair4', 'least_illustrative', 'most_illustrative'])
/tmp/ipykernel_4556/881429521.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  answers = pd.read_csv(os.path.join(GPT4_path, 'Phase2Answers-{}_no_least_most_gpt.txt'.format(idx)), sep=', ', names=['pair1', 'pair2', 'pair3', 'pair4', 'least_illustrative', 'most_illustrative'])
/tmp/ipykernel_4556/881429521.py:13: ParserWarni

In [4]:
data.head()

,pair1,pair2,pair3,pair4,least_illustrative,most_illustrative,user_selected_relation
0,house:bricks,lawn:grass,sock:thread,meal:food,sock:thread,lawn:grass,X is made of / is comprised of Y
1,house:bricks,lawn:grass,sock:thread,meal:food,meal:food,house:bricks,X is made of / is comprised of Y
2,house:bricks,lawn:grass,sock:thread,meal:food,meal:food,house:bricks,X is made of / is comprised of Y
3,house:bricks,lawn:grass,sock:thread,meal:food,lawn:grass,meal:food,X is made of / is comprised of Y
4,house:bricks,lawn:grass,sock:thread,meal:food,sock:thread,house:bricks,X is made of / is comprised of Y


In [5]:
def calc_rating_scores(df):

    raveled_pairs = df[['pair1', 'pair2', 'pair3', 'pair4']].values.ravel()
    all_pairs = np.unique(raveled_pairs)
    pair2least = {pair: (df.least_illustrative == pair).sum() for pair in all_pairs}
    pair2most = {pair: (df.most_illustrative == pair).sum() for pair in all_pairs}
    pair2num = {pair: (raveled_pairs == pair).sum() for pair in all_pairs}

    rating_scores = {}
    pairs = list(pair2num.keys())

    for pair in pairs:
        num_least = pair2least[pair]
        num_most = pair2most[pair]
        num_quest = pair2num[pair]
        pct_least = 100 * num_least / num_quest
        pct_most = 100 * num_most / num_quest
        score = pct_most - pct_least
        rating_scores[pair] = score

    return rating_scores

In [6]:
def calc_spearman(rel_data, tk_data):
    name_rel = rel_data.user_selected_relation.unique()[0]
    pair2gold = calc_rating_scores(tk_data[tk_data['user_selected_relation'] == name_rel])
    pair2test = calc_rating_scores(rel_data)

    cor = spearmanr(list(pair2gold.values()),
                    list(pair2test.values()))
    rho = cor.correlation
    pvalue = cor.pvalue

    return pd.DataFrame([{'rho': rho, 'pvalue': pvalue}])

In [7]:
spearman_df = answers.groupby('user_selected_relation').apply(lambda x: calc_spearman(x, data))
spearman_df

,,rho,pvalue
user_selected_relation,,,
X and Y are contrary / opposite to each other,0,0.357606,3.493601e-02
X is an expression that indicates Y,0,0.738815,5.234404e-08
X is made of / is comprised of Y,0,0.683619,4.360560e-07
X is the opposite direction from Y,0,0.796049,1.443225e-07
a Y is a part of an X,0,0.444778,2.486345e-03
a Y is one item in a collection/group of X,0,0.667393,1.383984e-06
an Y receives an X,0,0.583677,1.197870e-04
someone/something who is X cannot be Y or be in the state of Y,0,0.127958,4.022181e-01


In [8]:
spearman_coef = spearman_df.rho.mean()
spearman_coef

0.5499869250341645

In [9]:
num_cat_p5 = len(spearman_df[spearman_df.pvalue < 0.05])
num_cat_p5

7

In [10]:
num_cat_p5 = len(spearman_df[spearman_df.pvalue < 0.01])
num_cat_p5

6

## Across categories

In [38]:
cat_df = pd.read_csv(CAT_PATH, header=None, names=['num', 'let', 'cat', 'sub'])
cat_df['idx'] = cat_df['num'].astype(str) + cat_df['let'].apply(lambda x: x.replace(' ', ''))
cat_df = cat_df.merge(answers, on='idx', how='left')
cat_df = cat_df[['cat', 'user_selected_relation']].dropna().drop_duplicates()
cat_df = cat_df.merge(spearman_df.reset_index(), on='user_selected_relation', how='left')
cat_df = cat_df.groupby('cat').apply(lambda x: x['rho'].mean())

In [40]:
cat_df.round(3)

cat
 CASE RELATIONS     0.584
 CLASS-INCLUSION    0.667
 CONTRAST           0.577
 NON-ATTRIBUTE      0.128
 PART-WHOLE         0.564
 REFERENCE          0.739
dtype: float64

## No reversals

In [41]:
with open(REV_PATH, 'r') as f:
    reversals = f.read()

reversals = reversals.split('original:pair --> reversed:pair')[-1]

In [42]:
idxs = re.findall('[0-9]+[a-z]', reversals)
values = re.findall('--> [A-Z, a-z]+:[A-Z, a-z]+', reversals)
values = list(map(lambda x: x.split('--> ')[-1], values))

reversal_df = pd.DataFrame({'idxs': idxs, 'pair': values})
reversal_df['user_selected_relation'] = reversal_df['idxs'].map(rel_mapping)
reversal_df = reversal_df.dropna()
reversal_df = reversal_df.drop('idxs', axis=1)

In [43]:
def drop_reversals(question):
    question['drop'] = 0
    rel_name = question['user_selected_relation']
    pairs = question[['pair1', 'pair2', 'pair3', 'pair4']]
    revs = reversal_df[reversal_df.user_selected_relation == rel_name].pair

    if revs.isin(pairs).sum():
        question['drop'] = 1
    return question


In [44]:
data_no_rev = data.apply(lambda x: drop_reversals(x), axis=1)
data_no_rev = data_no_rev[data_no_rev['drop'] == 0].reset_index(drop=True)
data_no_rev.drop('drop', axis=1, inplace=True)

answers_no_rev = answers.apply(lambda x: drop_reversals(x), axis=1)
answers_no_rev = answers_no_rev[answers_no_rev['drop'] == 0].reset_index(drop=True)
answers_no_rev.drop('drop', axis=1, inplace=True)


In [17]:
spearman_no_rev_df = answers_no_rev.groupby('user_selected_relation').apply(lambda x: calc_spearman(x, data_no_rev))
spearman_no_rev_df

,,rho,pvalue
user_selected_relation,,,
X and Y are contrary / opposite to each other,0,0.353619,0.055237
X is an expression that indicates Y,0,0.636437,0.000040
X is made of / is comprised of Y,0,0.678638,0.000003
X is the opposite direction from Y,0,0.732525,0.000031
a Y is a part of an X,0,0.485565,0.001728
a Y is one item in a collection/group of X,0,0.625186,0.000035
an Y receives an X,0,0.628347,0.000090
someone/something who is X cannot be Y or be in the state of Y,0,0.171957,0.288694


In [18]:
spearman_no_rev_coef = spearman_no_rev_df.rho.mean()
spearman_no_rev_coef

0.5390341494360579

In [19]:
num_cat_p5_no_rev = len(spearman_no_rev_df[spearman_no_rev_df.pvalue < 0.05])
num_cat_p5_no_rev

6

In [20]:
num_cat_p5_no_rev = len(spearman_no_rev_df[spearman_no_rev_df.pvalue < 0.01])
num_cat_p5_no_rev

6

## RSME

In [72]:
pair2gold = calc_rating_scores(data)
pair2test = calc_rating_scores(answers)

pair2gold_no_rev = calc_rating_scores(data_no_rev)
pair2test_no_rev = calc_rating_scores(answers_no_rev)

In [78]:
range_rev_df = reversal_df.copy()
range_rev_df['original_pair'] = range_rev_df['pair'].apply(lambda x: x.split(':')[1] + ':' + x.split(':')[0])
range_rev_df['test_score'] = range_rev_df['original_pair'].apply(lambda x: pair2test[x])
range_rev_df['gold_score'] = range_rev_df['pair'].apply(lambda x: pair2gold[x])
range_rev_df['SE'] = range_rev_df.apply(lambda x: (x['test_score'] - x['gold_score']) ** 2, axis=1)
rmse_score = np.sqrt(range_rev_df['SE'].mean())

In [79]:
rmse_score

59.157206209846436